# Tutorial for Bokeh client visualization
* it is aimed to be used for develoopers to test feature of bokeh interactive  - standalone plotting

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import bokehDrawSA
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata
import pandas as pd
import numpy as np
import math
output_notebook()


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

- Create a random data source

In [ ]:
#init test random data
df = pd.DataFrame(np.random.random_sample(size=(1000, 6)), columns=list('ABCDEF'))
initMetadata(df)
mapDDC={0:"A0",1:"A1",2:"A2",3:"A3",4:"A4"}
df["B"]=np.linspace(0,1,1000)
df.eval("Bool=A>0.5", inplace=True)
df.eval("BoolB=B>0.5", inplace=True)
df.eval("BoolC=C>0.1", inplace=True)
df["A"]=df["A"].round(3);
df["B"]=df["B"].round(3);
df["C"]=df["C"].round(3);
df["D"]=df["D"].round(3);
df["AA"]=((df.A*10).round(0)).astype(pd.CategoricalDtype(ordered=True))
df["CC"]=((df.C*5).round(0)).astype(int)
df["DD"]=((df.D*4).round(0)).astype(int)
df["DDC"]=((df.D*4).round(0)).astype(int).map(mapDDC)
df["EE"]=(df.E*4).round(0)
df['errY']=df.A*0.02+0.02;
df.loc["A", 15] = math.nan
df.head(10)
df.meta.metaData = {'A.AxisTitle': "A (cm)", 'B.AxisTitle': "B (cm/s)", 'C.AxisTitle': "C (s)", 'D.AxisTitle': "D (a.u.)", 'Bool.AxisTitle': "A>half", 'E.AxisTitle': "Category"}


-Identify the layout

In [ ]:
figureArray = [
#   ['A'], ['C-A'], {"color": "red", "size": 7, "colorZvar":"C", "filter": "A<0.5"}],
    [['A'], ['A*A-C*C'], {"color": "red", "size": 2, "colorZvar": "A", "varZ": "C"}],
    [['A'], ['C+A', 'C-A', 'A/A']],
    [['B'], ['C+B', 'C-B'], { "colorZvar": "D", "errY": "errY", "rescaleColorMapper": True}],
    [['D'], ['(A+B+C)*D'], {"colorZvar": "D", "errY": "errY"} ],
    [['D'], ['D*10'], {"errY": "errY"}],
    {"size":2, "legend_options": {"label_text_font_size": "13px"}}
]
layout = {
    "A": [
        [0, 1, 2, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
        ],
    "B": [
        [3, 4, {'commonX': 1, 'y_visible': 3, 'x_visible':1, 'plot_height': 100}],
        {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2}
        ]
}
tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
pAll = bokehDrawArray(df, "A>0", figureArray, layout=layout, color="blue", size=4, tooltips=tooltips)
show(pAll[0])

In [ ]:
logging.disable(1)

-Draw the graphes

In [ ]:
output_file("test2.html")

widgets="slider.A(0,100,0.5,10,90),slider.B(0,100,0.5,10,90),slider.C(0,100,0.5,10,90),slider.D(0,100,0.5,10,90)"
    
figure=bokehDrawSA.fromArray(df,"A>0",figureArray, widgets, layout=figureLayout, color="blue", size=4, tooltips=tooltips)

In [ ]:
bokehDrawSA.initWidgets?
